# Monte Carlo

Monte Carlo in RL refers to a technique used to estimate the V(s) of a given policy (on-policy).

It can be summed up as: "How good is to be in a given state based on its future cumulative reward for a given policy? I will calculate the mean returns for every episode that goes through s."

In [3]:
from utils import compress_state, generate_extreme_value_state_image
import numpy as np

import minari
from collections import defaultdict
from IPython.display import HTML


def monte_carlo_evaluation(dataset_id, gamma=0.99):
    """
    Estimates the state-value function V(s) for a given policy using Monte Carlo first-visit evaluation.

    Parameters
    ----------
    dataset_id : str
        The ID of a Minari dataset, assumed to be generated by following a fixed policy in a known environment.
        Each episode in the dataset is treated as an independent rollout under this policy.
    
    gamma : float, optional (default=0.99)
        Discount factor used to calculate the return G_t = r_t + gamma * r_{t+1} + gamma^2 * r_{t+2} + ...

    Returns
    -------
    V : dict
        A dictionary mapping each encountered state (compressed as a hashable key from image, mission, and direction)
        to its estimated value V(s), computed as the average of returns following the first visit to that state
        across all episodes in the dataset.

    Methodology
    -----------
    This function uses **first-visit Monte Carlo evaluation**, meaning that for each episode, only the **first**
    occurrence of each state is used to compute its return. If a state appears multiple times within an episode,
    only the first encounter contributes to the value estimate.

    The rationale behind first-visit MC:
    - Prevents overcounting correlated returns from repeated visits within the same trajectory.
    - More robust in environments or policies where states may be revisited often due to loops or deterministic behavior.

    Assumptions & Implications
    --------------------------
    - The policy used to generate the dataset is fixed (though not necessarily deterministic).
    - The environment may be stochastic or deterministic — this method does not assume determinism.
    - This method provides an unbiased estimate of V(s) in the limit of many episodes, under standard MC assumptions.

    If you want to use **every-visit Monte Carlo** instead (i.e., update V(s) for every occurrence of s in an episode),
    simply remove the `visited` set and update the returns for every time step where state s appears.

    Comparison: First-Visit vs Every-Visit Monte Carlo
    --------------------------------------------------
    Every-Visit Monte Carlo — Pros:
    - ✅ More sample-efficient: Updates V(s) at every occurrence of a state, not just the first per episode.
    - ✅ Faster convergence in small datasets by making better use of available data.
    - ✅ Better suited to stochastic policies or environments where state re-visits yield diverse outcomes.
    - ✅ Lower variance early in learning due to averaging over more samples.

    Every-Visit Monte Carlo — Cons:
    - ❌ Can lead to overcounting in deterministic or loop-heavy environments.
    - ❌ May introduce bias when repeated visits in a trajectory provide highly correlated returns.
    - ❌ Less robust in structured or low-entropy environments where state visitation is redundant.

    Use first-visit MC when you want stable, conservative estimates that are less sensitive to trajectory-specific artifacts.
    Use every-visit MC when you need more aggressive learning from limited data, especially in noisy or high-entropy settings.

    Notes
    -----
    - The state is defined as a combination of symbolic image, mission text, and agent direction, as extracted from
      the observation dictionary.
    - The function assumes that the symbolic image is sufficient to index the state when combined with mission and direction.

    See Also
    --------
    Sutton & Barto, Reinforcement Learning: An Introduction — Chapter 5 (Monte Carlo Methods)
    """
    dataset = minari.load_dataset(dataset_id)
    V = defaultdict(float)
    returns_sum = defaultdict(float)
    returns_count = defaultdict(int)
    state_locations = {}

    for episode_idx, episode in enumerate(dataset.iterate_episodes()):
        episode_rewards = episode.rewards
        visited = set() # set of states already visited in this episode
        G = 0

        for t in reversed(range(len(episode_rewards))):
            observation = {k: v[t] for k, v in episode.observations.items()}
            state_key = compress_state(observation)
            reward = episode_rewards[t]
            G = gamma * G + reward # total return calculated as discounted sum of rewards

            if state_key not in visited:
                visited.add(state_key)
                returns_sum[state_key] += G
                returns_count[state_key] += 1
                V[state_key] = returns_sum[state_key] / returns_count[state_key]

                # Track where the state was first seen in reverse
                if state_key not in state_locations:
                    state_locations[state_key] = (episode_idx, t)

    values = np.array(list(V.values()))
    print("Value function statistics:")
    print(f"  Count:       {len(values)}")
    print(f"  Min value:   {np.min(values):.4f}")
    print(f"  Max value:   {np.max(values):.4f}")
    print(f"  Mean value:  {np.mean(values):.4f}")
    print(f"  Std dev:     {np.std(values):.4f}")

    return V, state_locations

In [ ]:

def monte_carlo_evaluation_every_visit(dataset_id, gamma=0.99):
    """
    Estimates the state-value function V(s) for a given policy using Monte Carlo every-visit evaluation.

    Parameters
    ----------
    dataset_id : str
        The ID of a Minari dataset, assumed to be generated by following a fixed policy in a known environment.
        Each episode in the dataset is treated as an independent rollout under this policy.
    
    gamma : float, optional (default=0.99)
        Discount factor used to calculate the return G_t = r_t + gamma * r_{t+1} + gamma^2 * r_{t+2} + ...

    Returns
    -------
    V : dict
        A dictionary mapping each encountered state (compressed as a hashable key from image, mission, and direction)
        to its estimated value V(s), computed as the average of returns following the first visit to that state
        across all episodes in the dataset.

    Methodology
    -----------
    This function uses **every-visit Monte Carlo evaluation**, meaning that for each episode, every occurrence
    of each state is used to compute its return. If a state appears multiple times within an episode,
    all occurrences contribute to the value estimate.
    The rationale behind every-visit MC:
    - Allows for more sample-efficient updates by leveraging all available data.
    - Better suited for environments where states may be revisited frequently, especially in stochastic settings.
    - Reduces variance in early learning by averaging over multiple samples of the same state.
    Assumptions & Implications
    --------------------------
    - The policy used to generate the dataset is fixed (though not necessarily deterministic).
    - The environment may be stochastic or deterministic — this method does not assume determinism.
    - This method provides an unbiased estimate of V(s) in the limit of many episodes, under standard MC assumptions.



    Comparison: First-Visit vs Every-Visit Monte Carlo
    --------------------------------------------------
    Every-Visit Monte Carlo — Pros:
    - ✅ More sample-efficient: Updates V(s) at every occurrence of a state, not just the first per episode.
    - ✅ Faster convergence in small datasets by making better use of available data.
    - ✅ Better suited to stochastic policies or environments where state re-visits yield diverse outcomes.
    - ✅ Lower variance early in learning due to averaging over more samples.

    Every-Visit Monte Carlo — Cons:
    - ❌ Can lead to overcounting in deterministic or loop-heavy environments.
    - ❌ May introduce bias when repeated visits in a trajectory provide highly correlated returns.
    - ❌ Less robust in structured or low-entropy environments where state visitation is redundant.

    Use first-visit MC when you want stable, conservative estimates that are less sensitive to trajectory-specific artifacts.
    Use every-visit MC when you need more aggressive learning from limited data, especially in noisy or high-entropy settings.

    Notes
    -----
    - The state is defined as a combination of symbolic image, mission text, and agent direction, as extracted from
        the observation dictionary.
    - The function assumes that the symbolic image is sufficient to index the state when combined with mission and direction.
    - This function is similar to `monte_carlo_evaluation`, but it updates the value for every occurrence of a state
        in an episode, rather than just the first visit.

    See Also
    --------
    Sutton & Barto, Reinforcement Learning: An Introduction — Chapter 5 (Monte Carlo Methods)
    """
    dataset = minari.load_dataset(dataset_id)
    V = defaultdict(float)
    returns_sum = defaultdict(float)
    returns_count = defaultdict(int)
    state_locations = {}

    for episode_idx, episode in enumerate(dataset.iterate_episodes()):
        episode_rewards = episode.rewards
        visited = set() # set of states already visited in this episode
        G = 0

        for t in reversed(range(len(episode_rewards))):
            observation = {k: v[t] for k, v in episode.observations.items()}
            state_key = compress_state(observation)
            reward = episode_rewards[t]
            G = gamma * G + reward # total return calculated as discounted sum of rewards

            if state_key not in visited:
                visited.add(state_key)
                returns_sum[state_key] += G
                returns_count[state_key] += 1
                V[state_key] = returns_sum[state_key] / returns_count[state_key]

                # Track where the state was first seen in reverse
                if state_key not in state_locations:
                    state_locations[state_key] = (episode_idx, t)

    values = np.array(list(V.values()))
    print("Value function statistics:")
    print(f"  Count:       {len(values)}")
    print(f"  Min value:   {np.min(values):.4f}")
    print(f"  Max value:   {np.max(values):.4f}")
    print(f"  Mean value:  {np.mean(values):.4f}")
    print(f"  Std dev:     {np.std(values):.4f}")

    return V, state_locations

In [4]:
def monte_carlo_incremental_evaluation(dataset_id, gamma=0.99, alpha=None):
    """
    Estimates the state-value function V(s) using First-Visit Incremental Monte Carlo Evaluation.

    Parameters
    ----------
    dataset_id : str
        ID of a Minari dataset generated by following a fixed policy.

    gamma : float, optional (default=0.99)
        Discount factor for computing return.
    alpha : float, optional (default=None)
        Step-size parameter for incremental averaging. If None, it is computed as 1 / N(s) where N(s) is the number of visits to state s.


    Returns
    -------
    V : dict
        A dictionary mapping compressed state keys to their estimated values.

    state_locations : dict
        A dictionary mapping state keys to the (episode index, time step) where they were first encountered.


    Methodology
    ----------
    This function implements **First-Visit Incremental Monte Carlo Evaluation**. It computes the value function V(s)
    by averaging the returns G_t for each state s, updating the value incrementally as new episodes are processed.

    The key steps are:
    1. For each episode, traverse the rewards in reverse order to compute the return G_t at each time step.
    2. Maintain a set of visited states to ensure only the first visit in each episode contributes to the value estimate.
    3. Use incremental averaging to update the value function V(s) for each state encountered.
    4. Store the first occurrence of each state in the dataset for reference.
    5. Return the final value function and the locations of each state.
    This method is particularly useful for environments where states may be revisited multiple times within an episode,
    as it avoids overcounting returns from repeated visits.


    Assumptions & Implications
    --------------------------
    - The dataset is generated by a fixed policy, ensuring that the state visitation distribution is stable.
    - The environment may be stochastic or deterministic; this method does not assume determinism.
    - The function assumes that the symbolic image, mission text, and agent direction together uniquely identify a state.
    - The value function V(s) converges to the true expected return for each state as the number of episodes increases.
    - This method provides an unbiased estimate of V(s) in the limit of many episodes, under standard MC assumptions.
        Assumptions & Implications

        
    If you want to use **every-visit Monte Carlo** instead (i.e., update V(s) for every occurrence of s in an episode),
    simply remove the `visited` set and update the returns for every time step where state s appears.

    
    Comparison with First-Visit Monte Carlo
    --------------------------------------------------
    First-Visit Incremental Monte Carlo — Pros:
    - ✅ More sample-efficient: Updates V(s) incrementally, allowing for faster convergence with fewer episodes.
    - ✅ Lower memory footprint: Does not require storing all returns for each state, only the current estimate.
    - ✅ Better suited for online learning scenarios where episodes are processed sequentially.
    - ✅ Provides a running estimate of the value function, which can be useful for real-time applications.
    
    First-Visit Incremental Monte Carlo — Cons:
    - ❌ May introduce bias if the state visitation distribution changes over time (e.g., in non-stationary environments).
    - ❌ Requires careful management of step-size parameters to ensure convergence. 
    - ❌ Less robust to high variance in returns compared to batch methods that average over multiple episodes.

    Use First-Visit Incremental MC when you need a fast, online estimate of the value function that can adapt to new data
    without requiring a full pass over all previous episodes. It is particularly useful in environments where states are revisited frequently,
    or when computational resources are limited and you need to maintain a running estimate of the value function.
    Notes
    -----
    - The state is defined as a combination of symbolic image, mission text, and agent direction, as extracted from
      the observation dictionary.
    - The function assumes that the symbolic image is sufficient to index the state when combined with mission and direction.
    - The function uses incremental averaging to update the value function, which is efficient for large datasets.
    - The first occurrence of each state is tracked to provide context for where the value was estimated in the dataset.

    See Also
    --------
    - Katerina Fragkiadaki, Monte Carlo Learning — Lecture 4, CMU 10-403
    """

    dataset = minari.load_dataset(dataset_id)
    V = defaultdict(float)
    N = defaultdict(int)  # visitation count for incremental averaging
    state_locations = {}

    for episode_idx, episode in enumerate(dataset.iterate_episodes()):
        episode_rewards = episode.rewards
        visited = set() # set of states already visited in this episode
        # visited set ensures First-Visit Monte Carlo: only the first occurrence of each state in an episode is used
        G = 0

        for t in reversed(range(len(episode_rewards))):
            observation = {k: v[t] for k, v in episode.observations.items()}
            state_key = compress_state(observation) # convert observation to a hashable state key: set of unique observations -> state
            reward = episode_rewards[t]
            G = gamma * G + reward

            if state_key not in visited:
                visited.add(state_key)
                N[state_key] += 1

                # Apply incremental averaging for the value function
                if alpha is None:
                    alpha = 1 / N[state_key]  # step-size for incremental average
                    
                V[state_key] += alpha * (G - V[state_key])

                if state_key not in state_locations:
                    state_locations[state_key] = (episode_idx, t)

    values = np.array(list(V.values()))
    print("Value function statistics:")
    print(f"  Count:       {len(values)}")
    print(f"  Min value:   {np.min(values):.4f}")
    print(f"  Max value:   {np.max(values):.4f}")
    print(f"  Mean value:  {np.mean(values):.4f}")
    print(f"  Std dev:     {np.std(values):.4f}")

    return V, state_locations


In [ ]:
dataset_id = "minigrid/BabyAI-Pickup/optimal-fullobs-v0"
output_path = "./minigrid/BabyAI-Pickup/optimal-fullobs-v0/montecarlo/highest_value_function.png"

generate_extreme_value_state_image(
    dataset_id=dataset_id,
    output_path=output_path,
    value_fn_generator=
        monte_carlo_evaluation,  # or monte_carlo_incremental_evaluation
    highest=True
)

# === Display in notebook ===
HTML(f'<img src="{output_path}" width="400">')

Value function statistics:
  Count:       51916
  Min value:   0.0977
  Max value:   0.9969
  Mean value:  0.5987
  Std dev:     0.2118


In [6]:
dataset_id = "minigrid/BabyAI-Pickup/optimal-fullobs-v0"
output_path = "./minigrid/BabyAI-Pickup/optimal-fullobs-v0/montecarlo/lowest_value_function.png"

generate_extreme_value_state_image(
    dataset_id=dataset_id,
    output_path=output_path,
    value_fn_generator=monte_carlo_evaluation,
    highest=False
)

# === Display in notebook ===
HTML(f'<img src="{output_path}" width="400">')

Value function statistics:
  Count:       51916
  Min value:   0.0977
  Max value:   0.9969
  Mean value:  0.5987
  Std dev:     0.2118
